# 📥 COLETA GITLAB - Etapa 2/6

## 📋 O que este notebook faz

Este notebook **baixa documentos do GitLab** para processamento local:

- 🔗 **Conecta ao GitLab** usando token de acesso
- 📂 **Lista arquivos** da pasta `30-Aprovados` recursivamente  
- 🔍 **Filtra tipos** suportados (.pdf, .docx, .md, .txt, .png, .jpg, .jpeg)
- 💾 **Baixa preservando** a estrutura original de pastas
- 📍 **Salva contexto** do commit atual para rastreabilidade

## 🎯 Configuração necessária

- `GITLAB_URL` - URL do repositório GitLab
- `GITLAB_ACCESS_TOKEN` - Token de acesso (obrigatório)
- `GITLAB_TARGET_FOLDER` - Pasta alvo (padrão: "30-Aprovados")

## 📊 Output esperado

~39 arquivos baixados (~0.3MB) em `pipeline-data/documents/` mantendo a estrutura de pastas.

---

## ⚙️ Configuração e Conexão

In [1]:
import os
import sys
from pathlib import Path
from urllib.parse import urlparse
import time
from datetime import datetime

# Marcar início da execução
stage_start = time.time()
start_timestamp = datetime.now().isoformat() + "Z"

# Adicionar src ao path para usar nossa biblioteca customizada
sys.path.insert(0, str(Path().parent / "src"))
from gitlab_client import NicGitlab

# Configuração
GITLAB_URL = os.getenv("GITLAB_URL", "http://gitlab.processa.info/nic/documentacao/base-de-conhecimento.git")
GITLAB_TOKEN = os.getenv("GITLAB_ACCESS_TOKEN")
GITLAB_BRANCH = os.getenv("GITLAB_BRANCH", "main")
TARGET_FOLDER = os.getenv("GITLAB_TARGET_FOLDER", "30-Aprovados")
EXTENSIONS = [".pdf", ".docx", ".md", ".txt", ".png", ".jpg", ".jpeg"]

if not GITLAB_TOKEN:
    raise ValueError("GITLAB_ACCESS_TOKEN é obrigatório")

# Extrair informações da URL
clean_url = GITLAB_URL.removesuffix(".git")
parsed = urlparse(clean_url)
instance_url = f"{parsed.scheme}://{parsed.netloc}"
project_path = parsed.path.lstrip("/")
branch = GITLAB_BRANCH

print(f"GitLab: {instance_url}")
print(f"Projeto: {project_path}")
print(f"Branch: {branch}")
print(f"Pasta: {TARGET_FOLDER}")

GitLab: http://gitlab.processa.info
Projeto: nic/documentacao/base-de-conhecimento
Branch: main
Pasta: 30-Aprovados


In [2]:
# Conectar ao GitLab usando nosso cliente customizado
gl = NicGitlab(instance_url, private_token=GITLAB_TOKEN)
print("✅ GitLab conectado ok!")

# Obter projeto diretamente por path (sem busca de ID)
project = gl.client.get_project(project_path)
print(f"✅ Projeto do GitLab carregado ok!")
print(f"ID do projeto: {project.project_id}")

# Obter commit atual
try:
    commits = project.commits.list(ref_name=branch, per_page=1)
    if commits:
        commit_hash = commits[0]["id"]  # É um dict, não objeto
        print(f"📍 Commit atual: {commit_hash[:8]}")
    else:
        commit_hash = "unknown"
        print("⚠️ Não foi possível obter commit")
except Exception as e:
    commit_hash = "unknown"
    print(f"⚠️ Erro ao obter commit: {e}")

# Criar diretório de destino
docs_dir = Path("pipeline-data/documents")
docs_dir.mkdir(parents=True, exist_ok=True)

# Limpar diretório
print(f"Limpando diretório: {docs_dir}")
for f in docs_dir.glob("*"):
    if f.is_file():
        f.unlink()
print(f"✅ Diretório limpo!")

print(f"Diretório preparado: {docs_dir}")

✅ GitLab conectado ok!
✅ Projeto do GitLab carregado ok!
ID do projeto: 266
📍 Commit atual: aeb81ff4
Limpando diretório: pipeline-data/documents
✅ Diretório limpo!
Diretório preparado: pipeline-data/documents


## 🔗 Conexão e Preparação

In [3]:
# Listar e filtrar arquivos
def list_files_recursive(project, path="", branch="main"):
    """Lista arquivos recursivamente em uma pasta"""
    files = []
    try:
        items = project.repository_tree(path=path, ref=branch, all=True)
        for item in items:
            if item["type"] == "blob":
                files.append({
                    "name": item["name"],
                    "path": item["path"],
                    "size": 0  # GitLab API não retorna tamanho na listagem
                })
    except Exception as e:
        print(f"Erro ao listar {path}: {e}")
    return files

# Listar arquivos na pasta alvo
print(f"Listando arquivos da pasta: {TARGET_FOLDER}")
all_files = list_files_recursive(project, TARGET_FOLDER)

# Filtrar por extensão
filtered_files = [
    {"name": i["name"], "path": i["path"], "size": 0}
    for i in project.repository_tree(path=TARGET_FOLDER, ref=branch, recursive=True, all=True, per_page=100)
    if i.get("type") == "blob" and any(i["name"].lower().endswith(ext.lower()) for ext in EXTENSIONS)
]

print(f"Arquivos encontrados: {len(all_files)}")
print(f"Arquivos filtrados: {len(filtered_files)}")

for f in filtered_files[:10]:  # Mostrar primeiros 10
    print(f"  {f['path']}")
if len(filtered_files) > 10:
    print(f"  ... e mais {len(filtered_files) - 10} arquivos")

Listando arquivos da pasta: 30-Aprovados


Arquivos encontrados: 1
Arquivos filtrados: 39
  30-Aprovados/Anexos/Imagens Self Checkout/FIgura 7 - Opção Menu.jpg
  30-Aprovados/Anexos/Imagens Self Checkout/Figura 1 – Interface inicial do Self Checkout.jpg
  30-Aprovados/Anexos/Imagens Self Checkout/Figura 2 – Interface de identificação do cliente.jpg
  30-Aprovados/Anexos/Imagens Self Checkout/Figura 3 – Interface para registro do produto.jpg
  30-Aprovados/Anexos/Imagens Self Checkout/Figura 4 - Msg produto na cesta.jpg
  30-Aprovados/Anexos/Imagens Self Checkout/Figura 5 - Escolher a forma de pagamento.jpg
  30-Aprovados/Anexos/Imagens Self Checkout/Figura 6 – Opção para Ajuda.jpg
  30-Aprovados/Anexos/Imagens Self Checkout/Figura 8 - Autenticacao do Menu.jpg
  30-Aprovados/Mapas/Processa Sistemas.md
  30-Aprovados/Mapas/Visão Geral do NIC.md
  ... e mais 29 arquivos


## 📂 Descoberta de Arquivos

In [4]:
# Baixar arquivos preservando a estrutura de pastas
downloaded = 0
errors = []
total_size = 0

for file_info in filtered_files:
    try:
        # Obter conteúdo do arquivo usando nosso cliente customizado
        content = project.download_file_raw(file_path=file_info["path"], ref=branch)

        # Caminho local com subpastas reproduzidas
        local_path = docs_dir / Path(file_info["path"])
        local_path.parent.mkdir(parents=True, exist_ok=True)

        # Salvar
        with open(local_path, "wb") as f:
            f.write(content)

        file_size = len(content)
        total_size += file_size
        downloaded += 1

        print(f"✅ {file_info['path']} ({file_size/1024:.1f} KB)")

    except Exception as e:
        errors.append(f"{file_info['path']}: {str(e)}")
        print(f"❌ {file_info['path']}: {str(e)}")

print(f"\n📊 Resumo:")
print(f"  Baixados: {downloaded}")
print(f"  Erros: {len(errors)}")
print(f"  Tamanho total: {total_size/1024/1024:.1f} MB")

if errors:
    print("\n❌ Erros:")
    for error in errors[:5]:
        print(f"  {error}")

✅ 30-Aprovados/Anexos/Imagens Self Checkout/FIgura 7 - Opção Menu.jpg (26.6 KB)


✅ 30-Aprovados/Anexos/Imagens Self Checkout/Figura 1 – Interface inicial do Self Checkout.jpg (26.9 KB)


✅ 30-Aprovados/Anexos/Imagens Self Checkout/Figura 2 – Interface de identificação do cliente.jpg (25.3 KB)
✅ 30-Aprovados/Anexos/Imagens Self Checkout/Figura 3 – Interface para registro do produto.jpg (37.9 KB)


✅ 30-Aprovados/Anexos/Imagens Self Checkout/Figura 4 - Msg produto na cesta.jpg (23.4 KB)
✅ 30-Aprovados/Anexos/Imagens Self Checkout/Figura 5 - Escolher a forma de pagamento.jpg (32.6 KB)


✅ 30-Aprovados/Anexos/Imagens Self Checkout/Figura 6 – Opção para Ajuda.jpg (23.7 KB)
✅ 30-Aprovados/Anexos/Imagens Self Checkout/Figura 8 - Autenticacao do Menu.jpg (27.1 KB)


...

✅ 30-Aprovados/Tópicos/Iniciar a compra.md (0.8 KB)
✅ 30-Aprovados/Tópicos/Operação contínua e evolução do sistema.md (2.3 KB)


✅ 30-Aprovados/Tópicos/Padrões de Documentação do NIC.md (5.2 KB)
✅ 30-Aprovados/Tópicos/Processo de documentação e validação.md (3.3 KB)


✅ 30-Aprovados/Tópicos/Propósito do NIC.md (4.2 KB)
✅ 30-Aprovados/Tópicos/Pré-requisitos técnicos.md (5.6 KB)


✅ 30-Aprovados/Tópicos/Registro de produtos.md (2.7 KB)
✅ 30-Aprovados/Tópicos/Reimpressão do último cupom.md (1.5 KB)


✅ 30-Aprovados/Tópicos/Retornar para registro de venda.md (2.9 KB)
✅ 30-Aprovados/Tópicos/Solicitação de ajuda.md (2.1 KB)

📊 Resumo:
  Baixados: 39
  Erros: 0
  Tamanho total: 0.3 MB


## 💾 Download e Armazenamento

In [5]:
# Salvar informação de commit para outros notebooks
import json

context_dir = Path("pipeline-data")
context_dir.mkdir(exist_ok=True)

context = {"gitlab_commit": commit_hash}

with open(context_dir / "context.json", "w") as f:
    json.dump(context, f, indent=2)

print(f"✅ Commit {commit_hash[:8]} salvo em pipeline-data/context.json")

✅ Commit e9c8a430 salvo em pipeline-data/context.json


## 📊 Relatório de Execução

In [6]:
# Calcular duração
stage_duration = time.time() - stage_start

# Carregar relatório existente
report_path = Path("pipeline-data/report.json")
if report_path.exists():
    with open(report_path, "r") as f:
        report = json.load(f)
else:
    report = {"stages": [], "context": {}, "summary": {}}

# Atualizar contexto com informações do GitLab
report["context"].update({
    "repository": project_path,
    "gitlab_url": instance_url,
    "branch": branch,
    "commit": commit_hash,
    "target_folder": TARGET_FOLDER
})

# Adicionar informações desta etapa
stage_report = {
    "stage": 2,
    "name": "Coleta GitLab",
    "status": "SUCCESS" if len(errors) == 0 else "FAILED",
    "start_time": start_timestamp,
    "duration_seconds": round(stage_duration, 2),
    "results": {
        "project_id": project.project_id,
        "files_found": len(filtered_files),
        "files_downloaded": downloaded,
        "download_errors": len(errors),
        "total_size_mb": round(total_size/1024/1024, 2)
    }
}

# Se houve erros, adicionar detalhes
if errors:
    stage_report["errors"] = errors[:5]  # Limitar a 5 erros

# Adicionar ou atualizar stage no relatório
stages_updated = False
for i, stage in enumerate(report["stages"]):
    if stage["stage"] == 2:
        report["stages"][i] = stage_report
        stages_updated = True
        break

if not stages_updated:
    report["stages"].append(stage_report)

# Atualizar timestamp
report["summary"]["last_update"] = datetime.now().isoformat() + "Z"

# Salvar relatório atualizado
with open(report_path, "w") as f:
    json.dump(report, f, indent=2, ensure_ascii=False)

print(f"📊 Relatório atualizado: {report_path}")
print(f"⏱️ Duração da etapa: {stage_duration:.2f}s")

📊 Relatório atualizado: pipeline-data/report.json
⏱️ Duração da etapa: 22.45s
